## Partition Pandas dataframe based on sentinel value

First create some sample data - simulate what is retrieved from database

In [1]:
import random, datetime
import pandas

d = {'timestamp': [], 'motor_rpm': []}
now = datetime.datetime.utcnow()
for i in range(1000):
    rpm = random.randrange(100,1500, 10)
    d['timestamp'].append(now)
    d['motor_rpm'].append(rpm)
    now += datetime.timedelta(minutes=1)
    
# Insert 5 "events" where motor rpm goes to zero
# An "event" is where we see 1 or more contiguous readings of zero rpm.
# i.e., if the motor stops - it could be down for a minute or several minutes
zero_rpm_indices = [3,4,
                   299,
                   466, 467, 468, 469,
                   700, 701,
                   950, 951, 952, 953, 954]
for idx in zero_rpm_indices:
    d['motor_rpm'][idx] = 0

df = pandas.DataFrame.from_dict(d)
df

,timestamp,motor_rpm
0,2021-03-03 15:52:28.312731,370
1,2021-03-03 15:53:28.312731,290
2,2021-03-03 15:54:28.312731,130
3,2021-03-03 15:55:28.312731,0
4,2021-03-03 15:56:28.312731,0
...,...,...
995,2021-03-04 08:27:28.312731,610
996,2021-03-04 08:28:28.312731,260
997,2021-03-04 08:29:28.312731,210
998,2021-03-04 08:30:28.312731,1260


## Partitioning

Now that we have a reasonable facsimile of the sensor data, the problem is identifying partition boundaries.

The 0 values in `motor_rpm` are our sentinels - but note we may have a continguous series of the sentinel value/

For my purposes, I define a partition as:
  1. Starting at the first non-zero RPM value following a 0 rpm record. In the case the first record is non-zero RPM, that will also begin a partition
  2. Ending at the last occurrence of a zero RPM value before a non-zero RPM value. In all cases, the last record ends the last partition.

In [2]:
# Step 1: Find the zero-rpm events
zero_rpm = df.loc[df['motor_rpm'] == 0]
zero_rpm

,timestamp,motor_rpm
3,2021-03-03 15:55:28.312731,0
4,2021-03-03 15:56:28.312731,0
299,2021-03-03 20:51:28.312731,0
466,2021-03-03 23:38:28.312731,0
467,2021-03-03 23:39:28.312731,0
468,2021-03-03 23:40:28.312731,0
469,2021-03-03 23:41:28.312731,0
700,2021-03-04 03:32:28.312731,0
701,2021-03-04 03:33:28.312731,0
950,2021-03-04 07:42:28.312731,0


In [3]:
# Step 2: Create a map of indexes to zero-rpm records
# i.e., we'll see what partition each zero-rpm event 'belongs' to.
m = zero_rpm.index.to_series().diff().ne(1).cumsum()
m

3      1
4      1
299    2
466    3
467    3
468    3
469    3
700    4
701    4
950    5
951    5
952    5
953    5
954    5
dtype: int64

In [4]:
# Step 3: Walk the map elements and use that to definepart partition start, stop tuples
def partition(df):
    partitions = []
    current_event = 1
    partition_start_idx = 0
    partition_end_idx = 0
    for idx, event in m.items():
        if event != current_event:
            partitions.append((partition_start_idx, partition_end_idx))
            partition_start_idx = partition_end_idx + 1
            current_event = event
        partition_end_idx = idx
    
    # The last event falls outside the loop
    partitions.append((partition_start_idx, partition_end_idx))

    # Finally, capture all remaining records (if any) after the last event.
    if partition_end_idx < len(df.index) - 1:
        partitions.append((partition_end_idx + 1, len(df.index) - 1))
    
    return partitions

partition(df)

[(0, 4), (5, 299), (300, 469), (470, 701), (702, 954), (955, 999)]

### Check the edge cases

That appears to work, but let's check edge cases where our data has zero-rpm event at beginning and/or end of the dataframe as well.

In [5]:
df.at[0, 'motor_rpm'] = 0
df.at[len(df.index) - 1, 'motor_rpm'] = 0
df

,timestamp,motor_rpm
0,2021-03-03 15:52:28.312731,0
1,2021-03-03 15:53:28.312731,290
2,2021-03-03 15:54:28.312731,130
3,2021-03-03 15:55:28.312731,0
4,2021-03-03 15:56:28.312731,0
...,...,...
995,2021-03-04 08:27:28.312731,610
996,2021-03-04 08:28:28.312731,260
997,2021-03-04 08:29:28.312731,210
998,2021-03-04 08:30:28.312731,1260


In [6]:
partitions = partition(df)
partitions

[(0, 4), (5, 299), (300, 469), (470, 701), (702, 954), (955, 999)]

### Next steps

Now, I can analyze specific partitions as needed

In [7]:
df

,timestamp,motor_rpm
0,2021-03-03 15:52:28.312731,0
1,2021-03-03 15:53:28.312731,290
2,2021-03-03 15:54:28.312731,130
3,2021-03-03 15:55:28.312731,0
4,2021-03-03 15:56:28.312731,0
...,...,...
995,2021-03-04 08:27:28.312731,610
996,2021-03-04 08:28:28.312731,260
997,2021-03-04 08:29:28.312731,210
998,2021-03-04 08:30:28.312731,1260


In [8]:
for i, (start, stop) in enumerate(partitions):
    print(f"\nParition {i}: df[{start}:{stop + 1}]")
    part_df = df[start:stop + 1]
    print(part_df.describe())


Parition 0: df[0:5]
        motor_rpm
count    5.000000
mean    84.000000
std    128.179562
min      0.000000
25%      0.000000
50%      0.000000
75%    130.000000
max    290.000000

Parition 1: df[5:300]
         motor_rpm
count   295.000000
mean    815.457627
std     387.093272
min       0.000000
25%     500.000000
50%     810.000000
75%    1170.000000
max    1490.000000

Parition 2: df[300:470]
         motor_rpm
count   170.000000
mean    747.058824
std     418.921759
min       0.000000
25%     410.000000
50%     710.000000
75%    1070.000000
max    1490.000000

Parition 3: df[470:702]
         motor_rpm
count   232.000000
mean    778.879310
std     420.860326
min       0.000000
25%     430.000000
50%     770.000000
75%    1150.000000
max    1490.000000

Parition 4: df[702:955]
         motor_rpm
count   253.000000
mean    789.960474
std     427.263810
min       0.000000
25%     410.000000
50%     850.000000
75%    1170.000000
max    1480.000000

Parition 5: df[955:1000]
         